In [62]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [63]:
import docker

In [64]:
client = docker.from_env()

In [83]:
_image = "cluster-autoscaler"
print(_image)

cluster-autoscaler


In [84]:
# old_registry = "vcr.vngcloud.vn/81-vks-public"
old_registry = "registry.vngcloud.vn/public"
new_registry = "vcr.vngcloud.vn/public"

In [85]:
images = client.images.pull(f"{old_registry}/{_image}", all_tags=True)

In [86]:
for image in images:
    img = image.tags[0]
    new_image = img.replace(old_registry, new_registry)
    print(image.tags[0])
    image.tag(new_image)

registry.vngcloud.vn/public/cluster-autoscaler:v1.27.4
registry.vngcloud.vn/public/cluster-autoscaler:v1.26.5
registry.vngcloud.vn/public/cluster-autoscaler:v1.25.4
registry.vngcloud.vn/public/cluster-autoscaler:v1.24.4
registry.vngcloud.vn/public/cluster-autoscaler:v1.24.3


In [87]:
for image in images:
    img = image.tags[0]
    new_image = img.replace(old_registry, new_registry)
    print(f"Pushing... {new_image}")
    client.images.push(new_image)
    print(f"Pushed {new_image}\n")

Pushing... vcr.vngcloud.vn/public/cluster-autoscaler:v1.27.4
Pushed vcr.vngcloud.vn/public/cluster-autoscaler:v1.27.4

Pushing... vcr.vngcloud.vn/public/cluster-autoscaler:v1.26.5
Pushed vcr.vngcloud.vn/public/cluster-autoscaler:v1.26.5

Pushing... vcr.vngcloud.vn/public/cluster-autoscaler:v1.25.4
Pushed vcr.vngcloud.vn/public/cluster-autoscaler:v1.25.4

Pushing... vcr.vngcloud.vn/public/cluster-autoscaler:v1.24.4
Pushed vcr.vngcloud.vn/public/cluster-autoscaler:v1.24.4

Pushing... vcr.vngcloud.vn/public/cluster-autoscaler:v1.24.3
Pushed vcr.vngcloud.vn/public/cluster-autoscaler:v1.24.3



In [88]:
!docker image prune -a -f

Deleted Images:
untagged: registry.vngcloud.vn/public/cluster-autoscaler:v1.24.3
untagged: registry.vngcloud.vn/public/cluster-autoscaler@sha256:d362e852d1328a0ea0aa78e28816286b053a7780c563776afa1693979463b021
untagged: vcr.vngcloud.vn/public/cluster-autoscaler:v1.24.3
untagged: vcr.vngcloud.vn/public/cluster-autoscaler@sha256:d362e852d1328a0ea0aa78e28816286b053a7780c563776afa1693979463b021
deleted: sha256:3013a318ee533ba64633e6c9b9ac83c2bec23d2a98532be60f8c1b8fc95fd26a
deleted: sha256:5e4c621b0ec54741a1093d9e441423fdb35865ac38151210ae3799708d112b98
untagged: registry.vngcloud.vn/public/cluster-autoscaler:v1.25.4
untagged: registry.vngcloud.vn/public/cluster-autoscaler@sha256:5bce951dad1823c252a90a8669d2172e64cbea5ccae7b4654e39414463a4cd40
untagged: vcr.vngcloud.vn/public/cluster-autoscaler:v1.25.4
untagged: vcr.vngcloud.vn/public/cluster-autoscaler@sha256:5bce951dad1823c252a90a8669d2172e64cbea5ccae7b4654e39414463a4cd40
deleted: sha256:e2e1285fbfd73b1505465b97f1dfe71900ba8b949217698ffa